In [18]:
import numpy as np
from keras.applications import inception_v3
from keras.preprocessing import image
from scipy import ndimage
import imageio
from keras import backend as K;
import matplotlib.pyplot as plt;

In [2]:
#disable learning
K.set_learning_phase(0)
model = inception_v3.InceptionV3(weights='imagenet', include_top = False);

W0902 05:42:52.600682 11316 deprecation_wrapper.py:119] From C:\Users\serge\Anaconda3\envs\KerasEnv\lib\site-packages\keras\backend\tensorflow_backend.py:153: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0902 05:42:52.619684 11316 deprecation_wrapper.py:119] From C:\Users\serge\Anaconda3\envs\KerasEnv\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0902 05:42:52.621684 11316 deprecation_wrapper.py:119] From C:\Users\serge\Anaconda3\envs\KerasEnv\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0902 05:42:52.638682 11316 deprecation_wrapper.py:119] From C:\Users\serge\Anaconda3\envs\KerasEnv\lib\site-packages\keras\backend\tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.


In [3]:
#model.summary()

In [4]:
layer_contributuins = {
    'mixed2':0.2,
    'mixed3':3.0,
    'mixed4':2.0,
    'mixed5':1.5
}

In [5]:
layer_dict = dict([(layer.name, layer) for layer in model.layers])

loss = K.variable(0.)
for layer_name in layer_contributuins:
    coeff = layer_contributuins[layer_name]
    activation = layer_dict[layer_name].output
    
    scaling = K.prod(K.cast(K.shape(activation), 'float32'))
    loss = loss + coeff * K.sum(K.square(activation[:, 2: -2, 2: -2, :])) / scaling

In [6]:
dream = model.input
grads = K.gradients(loss, dream)[0]
grads /= K.maximum(K.mean(K.abs(grads)), 1e-7)
outputs = [loss, grads]
fetch_loss_and_grads = K.function([dream], outputs)

In [7]:
def eval_loss_and_grads(x):
    outs = fetch_loss_and_grads([x])
    loss_value = outs[0]
    grad_values = outs[1]
    return loss_value, grad_values

In [8]:
def gradient_ascent(x, iterations, step, max_loss=None):
    for i in range(iterations):
        loss_value, grad_values = eval_loss_and_grads(x)
        if max_loss is not None and loss_value > max_loss:
            break
        print('...Loss value at', i,':', loss_value)
        x += step * grad_values
    return x

In [9]:
def resize_image(image, size):
    img = np.copy(image)
    factors = (1, 
             float(size[0]) / img.shape[1],
             float(size[1]) / img.shape[2],
             1)
    return ndimage.zoom(img, factors, order=1)

In [10]:
def save_img(img, fname):
    print(fname)
    pil_img = depreprocess_image(np.copy(img))
    imageio.imwrite(fname, pil_img)

In [11]:
def preprocess_image(image_path):
    img = image.load_img(image_path)
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = inception_v3.preprocess_input(img)
    return img

In [12]:
def depreprocess_image(x):
    if K.image_data_format() == 'channels_first':
        x = x.reshape((3, x.shape[2], x.shape[3]))
        x = x.transponse((1, 2, 0))
    else:
        x = x.reshape((x.shape[1], x.shape[2], 3))
    
    x /= 2.
    x += 0.5
    x *= 255.
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [13]:
step = 0.1
num_octave = 3
octave_scale = 1.4
iterations = 20
max_loss = 10.
base_image_path = 'img.jpg'
img = preprocess_image(base_image_path)
original_shape = img.shape[1:3]
successive_shapes = [original_shape]

In [14]:
for i in range(1, num_octave):
    shape = tuple([int(dim / (octave_scale ** i)) for dim in original_shape])
    successive_shapes.append(shape)

In [15]:
successive_shapes = successive_shapes[ : : -1]
original_img = np.copy(img)
shrunk_original_img = resize_image(img, successive_shapes[0])

In [16]:
for shape in successive_shapes:
    print('Processing image shape', shape)
    img = resize_image(img, shape)
    img = gradient_ascent(img,
                         iterations = iterations,
                         step = step,
                         max_loss = max_loss)
    upscaled_shrunk_original_img = resize_image(shrunk_original_img, shape)
    same_size_original = resize_image(original_img, shape)
    lost_detail = same_size_original - upscaled_shrunk_original_img
    
    img += lost_detail
    shrunk_original_img = resize_image(original_img, shape)
    save_img(img, fname = 'dream_at_scale_' + str(shape[0])+'_' + str(shape[1]) + '.jpg')

save_img(img, fname = 'final_dream.png')

Processing image shape (191, 255)
...Loss value at 0 : 0.90522265
...Loss value at 1 : 1.2008617
...Loss value at 2 : 2.626522
...Loss value at 3 : 6.7686214
dream_at_scale_191_255.jpg
Processing image shape (267, 357)
dream_at_scale_267_357.jpg
Processing image shape (375, 500)


C:\Users\serge\Anaconda3\envs\KerasEnv\lib\site-packages\scipy\ndimage\interpolation.py:611: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)


dream_at_scale_375_500.jpg
final_dream.png
